In [4]:
from flask import Flask, request, jsonify
from twilio. twiml.messaging_response import MessagingResponse
import os
from twilio.rest import Client
import mysql.connector
from mysql.connector import Error
import database as mydb
import pandas as pd 
import textEx
import database

#connect_mysql
try:
    connection = mysql.connector.connect(
        host='quote.c9ac6sewqau0.ap-southeast-2.rds.amazonaws.com',
        database='quote',
        user='admin',
        password='admin123'
    )
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
except Error as e:
    print("Error while connecting to MySQL", e)


#connect EC2 instance
account_sid = 'AC79c2fe6511ac0c9a1c3881c384798e22'
auth_token = 'c9a5e13731974f3a5d548102fbd96629'
client = Client(account_sid, auth_token)

message = client.messages.create(
  from_='whatsapp:+14155238886',
  body='Whatsapp client activated',
  to='whatsapp:+85261520721'
)

print(message.sid)


Connected to MySQL Server version  8.0.35
SM3b785dadacf959322cb6fa270ced9f40


In [ ]:
app = Flask(__name__)

def _formatString(concatText:str,supplier:str,effectiveDate:str) -> tuple:
    
 #(productName, productTag, supplier, category, packing, origin, brand, effectiveDate, spec1, spec2, spec3, spec4, spec5, spec6, price, weightUnit, warehouse, notes)
    brand = textEx.getBrand(concatText)
    country = textEx.getCountry(concatText)
    product = textEx.getProduct(concatText)
    warehouse = textEx.getWarehoue(concatText)
    packing = textEx.getPacking(concatText)
    price = textEx.getPrice(concatText)
    weightUnit = textEx.getWeightUnit(concatText)
    category = textEx.getCategory(concatText)
    spec = textEx.getSpec(concatText)

    # Sort 'spec' in alphabetical order
    spec.sort()

    # Initialize variables with empty strings
    spec1 = spec2 = spec3 = spec4 = spec5 = spec6 = ""

    # Assign values from 'spec' to 'spec1' to 'spec6' based on the length of 'spec'
    if len(spec) > 0:
        spec1 = spec[0]
    if len(spec) > 1:
        spec2 = spec[1]
    if len(spec) > 2:
        spec3 = spec[2]
    if len(spec) > 3:
        spec4 = spec[3]
    if len(spec) > 4:
        spec5 = spec[4]
    if len(spec) > 5:
        spec6 = spec[5]
 

    product_detail = tuple(concatText,product, supplier, category, packing,country,brand, effectiveDate,spec1 , spec2 , spec3 , spec4 , spec5 , spec6,price,weightUnit,warehouse,"")

    return product_detail

def _updateProduct(SQLconnection, productDetail:tuple ):
    database.update_product(SQLconnection,productDetail)
    return None


@app.route("/whatsapp", methods=['POST'])
def receive_whatsapp_message():
    # Extracting the message SID, sender's number, and message body from the request
    message_sid = request.form.get('MessageSid', '')
    sender = request.form.get('From', '')
    message_body = request.form.get('Body', '')

    print(f"Message SID: {message_sid}, From: {sender}, Message: {message_body}")

    # Converting the message to a string (note: it's already a string)
    message_str = str(message_body)

    # You can process the message string here
    print(f"Received Message: {message_str}")
    resp = MessagingResponse()


    #if read 報價 do sql
    if ("有無報價單" in message_str):
        query = "select * from Tradeasy_quotation"
        df = mydb.query_data_dataframe(connection, query)
        resp.message(df[['productName', 'price', 'weightUnit', 'warehouse']].to_string(index=False))


    ### message recieved
  
    # identify pdf or whatsapp quotation
    

    #1 (whatsapp) break down to individual rows

    #2 extract data from whatsapp message
    formattedData = _formatString(message_str)


    

    #4 ask user if the data formatted is ok
    user_verified = False

    #5 if yes update database

    if user_verified :
        
        for product in formattedData:   
            _updateProduct(connection,formattedData)
            resp.message("Quotation Updated")
    else:
        #6 if no return please check your data again
        resp.message("Please check your data again")


    return str(resp)


In [9]:
if __name__ == '__main__':
    app.run(host="0.0.0.0",debug=False,port=5000)

SMff918bdd4929ca6d2161e7a30616a13d
